<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from numba import njit
from utils import gen_GBM
from talib import BBANDS

@njit
def crossup(x, y, i):
    if x[i - 1] < y[i - 1] and x[i] > y[i]:
        return 1
    else:
        return 0

@njit
def crossdown(x, y, i):
    if x[i - 1] > y[i - 1] and x[i] < y[i]:
        return 1
    else:
        return 0

price = gen_GBM(120)
upper, middle, lower = BBANDS(price, 20, 1, 1)
price = price[20:]
upper = upper[20:]
middle = middle[20:]
lower = lower[20:]

In [2]:
def ret_np(price, upper, middle, lower, comission = .0003):
    L = price.shape[0]
    out = np.empty(L)
    out[:2] = 0.0
    out[-2:] = 0.0
    i = 1 # we need history to check crossup

    while i < (L-2):
        
        out[i+1] = 0.0
        
        if crossup(price, lower, i) and i < (L-2):
            out[i+1] = price[i+2]/(price[i+1]*(1+comission)) - 1.0
            trade = True
            while trade and i < (L-3):
                i += 1
                out[i+1] = price[i+2]/price[i+1] - 1.0
                if crossup(price, middle,i) or crossdown(price, lower, i):
                    trade = False
                    out[i+1] = (1-comission)*price[i+2]/price[i+1] - 1.0
        
        
        if crossdown(price, upper, i) and i < (L-2):
            out[i+1] = (1-comission)*price[i+1]/price[i+2] - 1.0
            trade = True
            while trade and i < (L-3):
                i += 1
                out[i+1] = price[i+1]/price[i+2] - 1.0
                if crossup(price, upper, i) or crossdown(price, middle, i):
                    trade = False
                    out[i+1] = price[i+1]/(price[i+2]*(1+comission)) - 1.0
         
        
        i +=1
    return out

In [8]:
@njit
def BBANDS_return(price, upper, middle, lower, comission = .0003):
    L = price.shape[0]
    out = np.empty(L)
    out[:] = 99.99
    out[:2] = 0.0
    out[-2:] = 0.0
    i = 1

    while i < (L-2):
        
        out[i+1] = 0.0
        
        if crossup(price, lower, i) and i < (L-2):
            out[i+1] = price[i+2]/(price[i+1]*(1+comission)) - 1.0
            trade = True
            while trade and i < (L-3):
                i += 1
                out[i+1] = price[i+2]/price[i+1] - 1.0
                if crossup(price, middle,i) or crossdown(price, lower, i):
                    trade = False
                    out[i+1] = (1-comission)*price[i+2]/price[i+1] - 1.0
        
        
        if crossdown(price, upper, i) and i < (L-2):
            out[i+1] = (1-comission)*price[i+1]/price[i+2] - 1.0
            trade = True
            while trade and i < (L-3):
                i += 1
                out[i+1] = price[i+1]/price[i+2] - 1.0
                if crossup(price, upper, i) or crossdown(price, middle, i):
                    trade = False
                    out[i+1] = price[i+1]/(price[i+2]*(1+comission)) - 1.0
         
        
        i +=1
    return out

price = gen_GBM(100000, random_seed=None)
upper, middle, lower = BBANDS(price, 20, 1, 1)
price = price[20:]
upper = upper[20:]
middle = middle[20:]
lower = lower[20:]

In [4]:
rnp = ret_np(price, upper, middle, lower)
%timeit ret_np(price, upper, middle, lower)

94.1 ms ± 334 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
rnb = BBANDS_return(price, upper, middle, lower)
%timeit BBANDS_return(price, upper, middle, lower)

564 µs ± 280 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
np.allclose(rnp, rnb)

True

In [7]:
for _ in range(1000):
    price = gen_GBM(100000, random_seed=None)
    upper, middle, lower = BBANDS(price, 20, 1, 1)
    price = price[20:]
    upper = upper[20:]
    middle = middle[20:]
    lower = lower[20:]
    rnp = ret_np(price, upper, middle, lower)
    rnb = BBANDS_return(price, upper, middle, lower)
    assert np.allclose(rnp, rnb), "Really?!"

In [9]:
for _ in range(10000):
    price = gen_GBM(1000, random_seed=None)
    upper, middle, lower = BBANDS(price, 20, 1, 1)
    price = price[20:]
    upper = upper[20:]
    middle = middle[20:]
    lower = lower[20:]
    if any(BBANDS_return(price, upper, middle, lower) == 99.99):
        print("Fail!")